In [1]:
import pandas as pd
import time
import geopandas
from glob import glob
from shapely.ops import cascaded_union
import pickle
import matplotlib.pyplot as plt

In [3]:
bids = pd.read_csv("data/all_bids.csv")[["round", "bidder", "tier", "latency", "census_id"]]
bids["state"] = bids["census_id"]//10000000000
bids_unfiltered = bids
bids = bids.drop_duplicates(subset=["bidder", "census_id"])
bids

,round,bidder,tier,latency,census_id,state
0,1,Rainbow Communications LLC,Gigabit,Low,200050816001,20
1,1,Rainbow Communications LLC,Gigabit,Low,200050816002,20
2,1,Rainbow Communications LLC,Gigabit,Low,200050816003,20
3,1,Rainbow Communications LLC,Gigabit,Low,200050816005,20
4,1,Rainbow Communications LLC,Gigabit,Low,200050817002,20
...,...,...,...,...,...,...
4038929,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,271159503002,27
4038930,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,271159504001,27
4039029,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,270753701003,27
4039401,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,550539604003,55


In [5]:
name_map = {
    "CCO Holdings, LLC":"Charter Communications Inc",
    "Windstream Services LLC, Debtor-In-Possession":"Windstream Holdings, Inc.",
    "Connect Everyone LLC":"Starry, Inc.",
    "Frontier Communications Corporation, DIP":"Frontier Communications Corporation",
    "California Internet, L.P. dba GeoLinks":"GeoLinks",
    "Etheric Communications LLC":"Etheric Networks, Inc.",
    "Consolidated  Communications, Inc.":"Consolidated Communications, Inc.",
    "Frontier Communications Northwest, LLC":"Ziply Fiber",
    "Citynet West Virginia, LLC":"Citynet, LLC", 
    "Computer 5, Inc. d/b/a LocalTel Communications": "LocalTel Communications",
    "Armstrong Telephone Company - Northern Division": "Armstrong Holdings, Inc.",
    "Commnet Wireless, LLC": "ATN International, Inc.",
    "Mediacom Communications Corporation": "Mediacom Communications Corp.",
    "Continental Divide Electric Cooperative": "Continental Divide Electric Cooperative, Inc.",
    "South Arkansas Telephone Company": "South Arkansas Telephone Co.",
    "Digital Connections Inc. dba PRODIGI": "Digital Connections, Inc.",
    "Shenandoah Cable Television, LLC": "Shenandoah Telecommunications Company",
    "Direct Communications Rockland, Inc": "Direct Communications Rockland, Inc.",
    "Pine Belt Communications, Inc.":"Pine Belt Communications Co. Inc.",
    "Savage Communications":"Savage Communications Inc.",
    "Hamilton County Telephone Co-op":"Hamilton County Telephone Co-Op",
    "Hotwire Communications, Ltd":"Hotwire Communications Ltd.",
    "Altice USA, Inc.":"Altice",
    "TruVista Communications, Inc.":"TruVista Communications of Georgia, LLC",
    "Hawaii Dialogix Telecom LLC":"Hawaii Dialogix Telecom LLC",
    "Safelink Internet LLC":"Safelink Internet",
    "American Heartland":"Farmers Telephone Company",
    "Cellular Services LLC.":"Cellular Services LLC",
    "Scott County Telephone Cooperative, Inc.":"Scott County",
    "Micrologic Inc.":"Micrologic, Inc.",
    "Pioneer Wireless, Inc":"Pioneer Wireless, Inc.",
    "St. John Telco":"St. John Telephone, Inc.",
    "One Ring Networks, Inc":"One Ring Networks, Inc.",
    "Emery Telephone dba Emery Telcom":"Emery Telcom",
    "XIT Telecommunication & Technology":"XIT Rural Telephone Cooperative, Inc.",
    "Somerset Telephone Co., Inc.":"SOMERSET TELEPHONE COMPANY",
    "Siuslaw Broadband, LLC dba Hyak Technologies":"Siuslaw Broadband, LLC",
    "Minnesota Connections c/o Consolidated Tel Company":"Consolidated Telephone Company",
    "DoCoMo Pacific, Inc.":"Docomo Pacific",
    "Roseau Electric Cooperative, Inc.":"Roseau Electric Cooperative",
    "Custer Telephone Cooperative, Inc.":"Custer Telephone Cooperative Inc.",
    "Lakeland Communications Group, LLC":"Lakeland Communications, Inc.",
    "LigTel Communications, Inc.":"Ligtel Communications",
    "Hamilton Long Distance Company":"Hamilton.net, Inc.",
    "Allen's T.V. Cable Service, Inc.":"Allen's TV Cable Service, Inc.",
    "W. T. Services, Inc.":"W.T. Services, Inc.",
    "Heart of the Catskills Comm. Inc., dba MTC Cable":"MTC Cable",
    "Citizens Vermont Acquisition Corporation":"Citizens Telephone Company",
    "Nova Cablevision, Inc":"Nova Cablevision, Inc.",
    "yondoo Broadband LLC":"yondoo Broadband, LLC",
    "Baraga Telephone Company":"Baraga Telephone Company Inc.",
    "Miles Communications LLC":"Miles Communications, Inc.",
    "PVT NetWorks, Inc.":"Penasco Valley Telephone Cooperative, Inc.",
    "Yucca Telecommunications Systems, Inc.":"Yucca Telecom",
    "H&B Communication's, Inc.":"H&B Enterprises Inc.",
    "MARQUETTE-ADAMS TELEPHONE COOPERATIVE, INC.":"Marquette-Adams Telephone Cooperative, Inc.",
    "Pinpoint Bidding Coalition":"Pinpoint Holdings, Inc.",
    "Computer Techniques, Inc. dba CTI Fiber":"CTI Fiber",
    "Barry Technology Services, LLC":"Barry Technology Services",
    "Bloosurf, LLC":"Bloosurf",
    "St Paul Cooperative Telephone Association":"St Paul Coop Telephone Assoc",
    "Farmers Mutual Cooperative Telephone Company":"FARMERS MUTUAL TELEPHONE COMPANY",
    "NTS Communications, LLC":"NTS, Inc.",
    "Unified Communications Inc.":"Unified Communications, Inc.",
    "Peoples Communication, LLC.":"Peoples Communication, Inc.",
    "Wood County Telephone Company d/b/a Solarus":"Solarus",
    "Comcell Inc.":"Comcell, Inc.",
    "Taylor Telephone Coop., Inc. dba Taylor Telecom":"Taylor Telephone Cooperative, Inc.",
    "Plateau Telecommunications, Incorporated":"Plateau",
    "AMA Communications, LLC":"AMA Communicaitons, L.L.C.",
    "AirCell, Inc.":"AirCell",
    "Bays-ET Highspeed LLC":"Bays-ET Highspeed Internet LLC",
    "Big Bend Telecom LTD":"Big Bend Telephone Company, Inc.",
    "Blue Ridge Cable Technologies, Inc.":"Blue Ridge Cable Technologies, Inc.",
    "Cass Cable TV, Inc.":"Cass Cable TV, Inc.",
    "City of Wilson GreenLight":"City of Wilson",
    "Climax Telephone Company dba CTS Telecom, Inc.":"Climax Telephone Company",
    "Cumberland Telephone Company, Inc":"CUMBERLAND TELEPHONE COMPANY",
    "Cumby Telephone Coooertive, Inc.":"Cumby Telephone Coop., Inc.",
    "Data Stream Mobile Technoligies Inc.":"Data Stream",
    "Get Wireless Inc":"Getwireless.net",
    "Hartington Telecommunications Co., Inc,":"Hartington Telecommunications Co., Inc.",
    "Hilliary Communications Consortium":"Hilliary Communications, LLC",
    "Hillsboro Telephone Company,  Inc.":"Hillsboro Telephone Company",
    "Home Telephone ILEC, LLC":"Home Telephone ILEC, LLC d/b/a Home Telecom",
    "Hood Canal Telephone Co., Inc":"Hood Canal Communications",
    "Kingdom Telecommunications, Inc.":"Kingdom Telecommunications Inc",
    "Lincolnville Communications":"Lincolnville Telephone Company",
    "Local Internet Service Company, Inc.":"Local Internet Service Company",
    "Martell Enterprise  Inc.":"Martell Enterprises, Inc.",
    "Massena Telephone Company, Inc.":"Massena Telephone Company",
    "Moundridge Telephone Company":"Moundridge Telephone Co.",
    "MyServer.org, Inc dba San Diego Broadband":"San Diego Broadband",
    "Newbreak Management, LLC":"Newbreak Communications",
    "NexGenAccess Incorporated":"NexGenAccess",
    "Northwoods Communication Technologies, LLC":"Northwoodsconnect",
    "One Point Technologies Inc":"One Point Technologies Inc.",
    "Panhandle Telecommunication Systems, Inc.":"Panhandle Telephone Cooperative, Inc.",
    "Pathwayz Communications Inc":"Pathwayz Communications, Inc.",
    "RONAN TELEPHONE COMPANY":"Ronan Telephone Co",
    "Rainbow Communications LLC":"Rainbow Telecommunications Association, Inc.",
    "SOUTHWEST ARKANSAS TELEPHONE COOPERATIVE, INC.":"Southwest Arkansas Telephone Cooperative, Inc.",
    "South Central Wireless, Inc.":"South Central Wireless Inc.",
    "Southern Montana Telephone Company (SMTC)":"Southern Montana Telephone Company",
    "Steelville Telephone Exchange Inc.":"Steelville Telephone Exchange Inc",
    "Tekstar Communications, Inc. dba Arvig":"Arvig Enterprises, Inc.",
    "Upsala Cooperative Telephone Association dba Sytek":"UPSALA COOPERATIVE TELEPHONE ASSOCIATION",
    "Valley Telephone Cooperative, Inc":"Valley Telephone Cooperative, Inc.",
    "Velocity.Net Communications, Inc.":"Velocity Communications, Inc.",
    "Woodstock Telephone Co.":"Woodstock Telephone Company",
    "ZIRKEL Wireless, LLC":"Zirkel Wireless",
    "coon valley cooperative telephone association inc.":"Coon Valley Co-op Telephone Association, Inc.",
}

bids["bidder"] = bids["bidder"].map(lambda x: name_map[x] if x in name_map else x)
bids_unfiltered["bidder"] = bids_unfiltered["bidder"].map(lambda x: name_map[x] if x in name_map else x)
bids

/var/folders/18/gn_h02yd70g1wz5b9_6hcjjr0000gq/T/ipykernel_54531/2417878143.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids["bidder"] = bids["bidder"].map(lambda x: name_map[x] if x in name_map else x)


,round,bidder,tier,latency,census_id,state
0,1,"Rainbow Telecommunications Association, Inc.",Gigabit,Low,200050816001,20
1,1,"Rainbow Telecommunications Association, Inc.",Gigabit,Low,200050816002,20
2,1,"Rainbow Telecommunications Association, Inc.",Gigabit,Low,200050816003,20
3,1,"Rainbow Telecommunications Association, Inc.",Gigabit,Low,200050816005,20
4,1,"Rainbow Telecommunications Association, Inc.",Gigabit,Low,200050817002,20
...,...,...,...,...,...,...
4038929,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,271159503002,27
4038930,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,271159504001,27
4039029,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,270753701003,27
4039401,13,Consortium of AEG and Heron Broadband I,Gigabit,Low,550539604003,55


In [6]:
existing = pd.read_csv("data/existing_service.csv", encoding="latin")
existing["bg"] = existing["BlockCode"]//1000
existing  = existing.loc[(existing["MaxAdDown"]>=25) & (existing["MaxAdUp"]>=3) & (existing["TechCode"]!=60)]
existing["state"] = existing["bg"]//10000000000
existing['names'] = [frozenset(x) for x in zip(existing.ProviderName, existing.HoldingCompanyName, existing.DBAName, existing.HocoFinal)]
existing

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,bg,state,names
0,1,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501002019,70,1,40.0,10.0,1,410139501002,41,"(Webformix, Webformix Company)"
1,2,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139503002023,70,1,40.0,10.0,1,410139503002,41,"(Webformix, Webformix Company)"
2,3,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139502002027,70,1,40.0,10.0,1,410139502002,41,"(Webformix, Webformix Company)"
3,4,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501003004,70,1,40.0,10.0,1,410139501003,41,"(Webformix, Webformix Company)"
4,5,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139504001109,70,1,40.0,10.0,1,410139504001,41,"(Webformix, Webformix Company)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42151570,42151571,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003006,70,1,30.0,10.0,1,720054006003,72,"(Osnet Wireless Corp., OSNET Wireless Corp., O..."
42151571,42151572,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003005,70,1,30.0,10.0,1,720054006003,72,"(Osnet Wireless Corp., OSNET Wireless Corp., O..."
42151572,42151573,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003004,70,1,30.0,10.0,1,720054006003,72,"(Osnet Wireless Corp., OSNET Wireless Corp., O..."
42151573,42151574,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003003,70,1,30.0,10.0,1,720054006003,72,"(Osnet Wireless Corp., OSNET Wireless Corp., O..."


In [9]:
bidders = set(bids.bidder.unique())
all_names =  set(existing.names.unique())

In [19]:
from collections import defaultdict
from copy import deepcopy
found = defaultdict(set)
bidders_no_matching = deepcopy(bidders)
names_no_matching = deepcopy(all_names)

for bidder in bidders:
    for name in all_names:
        if bidder in name:
            bidders_no_matching.discard(bidder)
            names_no_matching.discard(name)
            found[bidder]|=set(name)

In [34]:
with open("../outputs/bidders_no_existing.txt", "w") as f:
    for bidder in bidders_no_matching:
        f.write(bidder+"\n")
        
with open("../outputs/existing_no_bidders.txt", "w") as f:
    for name in names_no_matching:
#         try:
            f.write(str(list(name))+"\n")
#         except:
#             print(name)
        
with open("../outputs/found.txt", "w") as f:
    for k, v in found.items():
        f.write(k+": "+str(v)+"\n")

In [26]:
for name in names_no_matching:
    print(", ".join(name))

Moore & Liberty Telephone Company/Griggs County Telephone, Griggs County Telephone Co, Griggs County Telephone Company
Campus Communications Group, Inc., Pavlov Media, Inc., Campus Communications Group
Fiber Wave, LLC
Monmouth Independence Network, MInet, Monmouth Independence Networks
Park Region Mutual Telephone Company, Park Region Telephone
LightSpeed Technologies LLC., LightSpeed Technologies LLC
Level 4 Telcom, Levan Town
NEXT, Powered by NAEC, LLC, Next, powered by NAEC, LLC, North Arkansas Electric Cooperative, Inc., Next, Powered by NAEC, LLC
Smithville Communications, Smithville Holding Company, Inc., Smithville Communications, Inc
MUSFiberNET, Morristown Utility Commission, Morristown Utility
Bluffton Tel. Co. Inc., Hargray of Georgia, Inc., Hargray Communications Group, Inc.
Alma Telecom, Inc., ATC BROADBAND LLC, ATC Broadband LLC
Lobo Internet Services Ltd, Lobo Internet Services, Ltd.
The Lincoln County Telephone System, Inc., Lincoln Communications, Inc., Lincoln County 

TypeError: sequence item 0: expected str instance, float found